<a href="https://colab.research.google.com/github/Bourbon-Rye/Baesian-Cropability/blob/main/PilipiNuts_2023_Baesian_Cropability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [3]:

# @Libraries
import numpy as np
import pandas as pd
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import json
import re
import functools

from pathlib import Path
from sklearn import preprocessing
from plotly.subplots import make_subplots
from scipy.stats import chi2_contingency
from sklearn.impute import SimpleImputer

In [4]:

# @Utilities
def read_csv_to_df(csvfile: Path, comment_symbol='#') -> pd.DataFrame:
    """Read CSV to DataFrame with comment validation.
        Allows comment lines in CSVs where line[0] == comment_symbol.
        Also removes newlines.
        Does not recognize comment_symbol anywhere else. 
    """
    tempfile = Path('temp.csv')
    with open(csvfile, 'r') as csv, open(tempfile, 'w+') as temp:
        lines = csv.readlines()
        for line in lines:
            if line[0] != comment_symbol:
                temp.write(line)
    return pd.read_csv(tempfile)

with open('datasets/region_provinces.json') as jsonfile:
    regions_provinces = json.load(jsonfile)['PHILIPPINES']
regions_provinces = {key.lower():regions_provinces[key] for key in regions_provinces}
regions = regions_provinces.keys()
provinces = set([item.lower() for key in regions for item in regions_provinces[key]])
regions = set(regions)

# NOTE: Fix for bad regions, thanks PSA
bad_regions = ['AONCR', 'BARMM', 'CAR', 'MIMAROPA', 'NCR',
           'Region 1', 'Region 2', 'Region 3', 'Region 4A',
           'Region 5', 'Region 6', 'Region 7', 'Region 8',
           'Region 9', 'Region 10', 'Region 11', 'Reg12', 'CARAGA']
region_mapping = {bad.lower():good.lower() for (bad,good) in zip(bad_regions, regions_provinces.keys())}
corrections = {
    "AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)": "bangsamoro autonomous region in muslim mindanao (barmm)",
    "autonomous region in muslim mindanao (armm)": "bangsamoro autonomous region in muslim mindanao (barmm)",
    "mimaropa region": "mimaropa region (mimaropa)"
}
region_mapping.update(corrections)

temp = [key.split('(')[1].rstrip(')') for key in regions_provinces]
region_short_to_short_long = {bad:good for (bad,good) in zip(temp, regions_provinces.keys())}
print(sorted(regions))
print(region_short_to_short_long)

['areas outside national capital region (aoncr)', 'bangsamoro autonomous region in muslim mindanao (barmm)', 'cordillera administrative region (car)', 'mimaropa region (mimaropa)', 'national capital region (ncr)', 'region i (ilocos region)', 'region ii (cagayan valley)', 'region iii (central luzon)', 'region iv-a (calabarzon)', 'region ix (zamboanga peninsula)', 'region v (bicol region)', 'region vi (western visayas)', 'region vii (central visayas)', 'region viii (eastern visayas)', 'region x (northern mindanao)', 'region xi (davao region)', 'region xii (soccsksargen)', 'region xiii (caraga)']
{'aoncr': 'areas outside national capital region (aoncr)', 'barmm': 'bangsamoro autonomous region in muslim mindanao (barmm)', 'car': 'cordillera administrative region (car)', 'mimaropa': 'mimaropa region (mimaropa)', 'ncr': 'national capital region (ncr)', 'ilocos region': 'region i (ilocos region)', 'cagayan valley': 'region ii (cagayan valley)', 'central luzon': 'region iii (central luzon)', '

In [5]:
def is_region(x: str, regions=regions) -> bool:
    x = x.strip(' .')
    x = x.lower()
    if x == "cagayan":
        return False
    for region in regions:
        if x in region:
            return True
    else:
        return False
    
def is_province(x: str, provinces=provinces) -> bool:
    x = x.strip(' .')
    x = x.lower()
    for province in provinces:
        if x in province:
            return True
    else:
        return False

def get_quarter_columns(df: pd.DataFrame, year_range: range, period_idx: int):
    """Assumes contiguous period columns and that columns before  are ID columns"""
    df_quarter = df.iloc[:, :period_idx]
    for year in year_range:
        for q in range(0, 12, 3):
            df_quarter[f"{year} Q{q//3+1}"] = df.filter(regex=str(year), axis=1).iloc[:, q:q+3].mean(axis=1)
    return df_quarter

def get_annual_columns(df: pd.DataFrame, year_range: range, period_idx: int):
    """Assumes contiguous period columns and that columns before year_start are ID columns"""
    df_annual = df.iloc[:, :period_idx]
    for year in year_range:
        df_annual[f"{year}"] = df.filter(regex=str(year), axis=1).mean(axis=1)
    return df_annual

def swap_columns(df: pd.DataFrame, col1: str, col2: str):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

def normalize(df: pd.DataFrame, col: str, minmax = True):
    """Can use mean normalization and minmax normalization."""
    tdf = df[col]
    if minmax:
        tdf = (tdf-tdf.min())/(tdf.max()-tdf.min())
    else:
        tdf = (tdf-tdf.mean())/tdf.std()
    df[col] = tdf
    return df

def drop_rows_with_zeros(df: pd.DataFrame, ref_col_idx: int, all_zeros=False):
    """Drop rows if some values are zeros, or if all values are zeros.
    Assumes contiguous reference columns, i.e. columns to use in deciding whether to drop."""
    return df[~(df.iloc[:, ref_col_idx:] == 0).all(axis=1)] if all_zeros else df[~(df.iloc[:, ref_col_idx:] == 0).any(axis=1)]

def dual_plot(df: pd.DataFrame, y1: str, y2: str,
              x_title: str, y1_title: str, y2_title: str, title_text: str):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    # Add traces
    fig.add_trace(
        go.Scatter(x=df.index, y=df[y1], name=y1),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df[y2], name=y2),
        secondary_y=True,
    )
    # Add titles
    fig.update_layout(title_text=title_text)
    fig.update_xaxes(title_text=x_title)
    # Set y-axes titles
    fig.update_yaxes(title_text=y1_title, secondary_y=False)
    fig.update_yaxes(title_text=y2_title, secondary_y=True)

    fig.show()

def show_heatmap(df: pd.DataFrame):
    fig = plt.figure(figsize=(15,10))
    sns.heatmap(df.corr(method='pearson'), annot = True, cmap="Blues")
    plt.title("Correlation Heatmap")
    plt.show()
    


In [6]:
# OTG Cleanup
# VALUE OF PRODUCTION (5/24/2024)
datapath = Path("datasets/agricultural-indicators/value-of-production/")
writepath = Path("datasets")

files = list(datapath.glob("*.csv"))
df = pd.DataFrame()
for file in files:
    with open(file) as f:
        region = f.readlines()[0].split(':')[0].lstrip("\"")
        if region == u"\ufeffTest\n":
            region = "Cordillera Administrative Region (CAR)"
        tdf = pd.read_csv(file, skiprows=1, na_values=".")
        tdf["Geolocation"] = region
        df = pd.concat([df, tdf])
df["Subsector"] = "Rice"
df.drop(columns=["Type of Valuation"], inplace=True)
df.rename({"Subsector": "Commodity"}, axis=1, inplace=True)

# Imputation of VOP df
imp = SimpleImputer(missing_values=pd.NA, strategy='mean')
imp.fit(df.iloc[:, 1:-1])
df[df.columns[1:-1]] = imp.transform(df.iloc[:, 1:-1])
df_val_of_prod = df

# Annualized rice stocks NOTE: Perhaps try to match this with the prices as this has monthly, for national only
df = read_csv_to_df("datasets/cropyield/stocks-palay-corn_yearly_1980-2024.csv")
df.dropna(inplace=True)
df = drop_rows_with_zeros(df, ref_col_idx=2, all_zeros=True)
df = df[df["Sector"] == "Rice: Total Stock"]
df["Sector"] = "Rice"
df = pd.concat([df.iloc[:, :2], df.mean(axis=1, numeric_only=True)], axis=1)
df.rename(columns={"Sector": "Commodity", "Year": "Period", 0: "Stocks"}, inplace=True)
df["Period"] = df["Period"].astype(object)
df = df[df["Period"].isin(range(2012, 2024))]
df_stocks = df

# Baesian Plots


## Yenzy Plots
H0.2: There is no significant relationship between market conditions and food crop production.

Goal: Visually and statistically assess the relationship between market indicators and crop yield.

Visual tests are dual plots and scatterplots. Statistics for relationship testing: contingency table and chi-square test.

**Tested Assessments in both Annual & Quarterly and Regional & Provincial**
| Cropyield Indicator | Market Indicator | Geolocation | Commodity
| --- | --- | --- | --- |
| Volume | Retail Price | Philippines | Rice | 
| Volume | Wholesale Price | Philippines | Rice | 
| Volume | Farmgate Price | Philippines | Rice | 

In [7]:
# Sample Relationship Test
data = [[207, 282, 241], [234, 242, 232]]
stat, p, dof, expected = chi2_contingency(data)
 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.10319714047309392
Independent (H0 holds true)


In [8]:
# Volume of Production of Palay and Corn across the Years (Quarterly and Annual)
df2 = read_csv_to_df("datasets/agricultural-indicators/Volume_Rice and Corn_quarterly.csv")
df2 = df2[df2["Geolocation"] == "PHILIPPINES"]
df2 = df2.melt(id_vars=["Commodity", "Geolocation"], value_vars=df2.columns[2:], var_name="Period", value_name="Volume")
fig = px.line(df2, x="Period", y="Volume", color="Commodity", title='Quarterly Volume of Production of Crops across the Years<br>National Averages in Metric Tons').update_layout(
    xaxis_title="Period", yaxis_title = "Volume of Production")
fig.show()

In [13]:
# Creating megadataset for rice
def preprocess_baesians_1(df: pd.DataFrame, commodity: str, rename_to: str, melt_value: str|None, regional=True,
                          impute=False):
    """Assumes Geolocation | Commodity | Period columns.
    Filters to regional if regional=True, else filters to provincial.
    """
    # df.dropna(inplace=True)
    df["Geolocation"] = df["Geolocation"].str.lstrip(".").str.lower()
    df["Geolocation"] = df["Geolocation"].replace(region_mapping)
    df = df[df["Geolocation"].apply(is_region)] if regional else df[df["Geolocation"].apply(is_province)]
    df = df[df["Commodity"] == commodity]
    df["Commodity"] = rename_to # Rename everything in Commodity to Rice
    if melt_value:
        df = df.melt(id_vars=["Geolocation", "Commodity"], value_vars=df.columns[2:], var_name="Period", value_name=melt_value)
    return df

# RICE: Volume and Retail Price (Quarterly-Regional)
# Volume of Rice
df1 = read_csv_to_df("datasets/agricultural-indicators/Volume_Rice and Corn_quarterly.csv")
df1 = preprocess_baesians_1(df1, "Palay", "Rice", melt_value="Volume")

# Retail Price of Rice
df2 = read_csv_to_df("datasets/prices/prices_retail_2012-2023.csv")
tdf1 = pd.read_csv("datasets/prices/prices_retail_caraga_supplement.csv", skiprows=1)
tdf2 = pd.read_csv("datasets/prices/prices_retail_car_supplement.csv", skiprows=1)
df2 = pd.concat([df2, tdf1, tdf2])
df2 = get_quarter_columns(df2, range(2012, 2024), 2)
df2 = preprocess_baesians_1(df2, "RICE, REGULAR-MILLED, 1 KG", "Rice", melt_value="Retail Price")

# Wholesale Price of Rice
df3 = pd.read_csv("datasets/prices/prices_wholesale-new-series_2010-2023.csv",)
df3 = get_quarter_columns(df3, range(2012, 2024), 2)
df3 = preprocess_baesians_1(df3, "Well Milled Rice (WMR)", "Rice", melt_value="Wholesale Price")

# Farmgate Price of Rice
df4 = read_csv_to_df("datasets/prices/prices_farmgate-new-series_2010-2023.csv")
df4 = get_quarter_columns(df4, range(2012, 2024), 2)
df4 = preprocess_baesians_1(df4, "Palay [Paddy] Other Variety, dry (conv. to 14% mc)", "Rice", melt_value="Farmgate Price")
df4.dropna(inplace=True)

# Cropyield (Area Harvested) of Rice
df = pd.read_csv("datasets/cropyield/cropyield-palay-corn_quarterly_2010-2023.csv", skiprows=1)
df = preprocess_baesians_1(df, "Palay", "Rice", melt_value="Area Harvested")
df = df[df["Period"].str.contains("Q")]

# Consumer Price Index (All Income) per Region of Rice
df6 = pd.read_csv("datasets/price-indices-2018-based/cpi_all-income-households-by-cg-with-backcasting_1994-2023.csv")
df6 = get_quarter_columns(df6, range(2012, 2024), 2)
df6 = preprocess_baesians_1(df6, "01.1.1.12 - Rice", "Rice", melt_value="CPI All Income")

# Consumer Price Index (Bottom 30) per Region of Rice
df7 = pd.read_csv("datasets/price-indices-2018-based/cpi_bottom-30-by-cg-with-backcasting_2012-2017.csv")
df7 = preprocess_baesians_1(df7, "01.1.1.12 - Rice", "Rice", melt_value=None)
tdf = pd.read_csv("datasets/price-indices-2018-based/cpi_bottom-30-by-cg_2018-2023.csv")
tdf = preprocess_baesians_1(tdf, "01.1.1.12 - Rice", "Rice", melt_value=None)
df7 = pd.merge(df7, tdf, on=["Geolocation", "Commodity"])
df7 = get_quarter_columns(df7, range(2012, 2024), 2)
df7 = preprocess_baesians_1(df7, "Rice", "Rice", melt_value="CPI Bottom 30")

# Value of Production of Rice (Annual)
df8 = df_val_of_prod
col = df8.pop("Geolocation")
df8.insert(0, col.name, col)
df8 = preprocess_baesians_1(df8, "Rice", "Rice", melt_value="Value of Production")

# Stocks of Rice (Annual)
df9 = df_stocks

# Merge all dfs into a single df NOTE: geolocation must be in lower case
# NOTE: barmm is removed because it has null values, unfortunately
dfs = [df1, df2, df3, df4, df, df6, df7]
df = functools.reduce(lambda left, right: pd.merge(left, right), dfs)

fig = px.scatter(df, x="Volume", y="Retail Price", color="Geolocation")
# fig.show()

# Adding Value of Production (Annual to Quarterly)
tdf = df.iloc[:,:3]
tdf["Period"] = df["Period"].str.replace(" Q\d", "", regex=True)
df["Value of Production"] = pd.merge(tdf, df8)["Value of Production"]
df.dropna(inplace=True)

# BUG: Adding Stocks (Annual to Quarterly)
tdf = pd.DataFrame(df["Period"].str.replace(" Q\d", "", regex=True))
df1 = tdf.iloc[1:2].reset_index(drop=True)
df2 = df9.iloc[0:1, 1:3].reset_index(drop=True)

print(df1, type(df1))
print(df2, type(df2))

pd.merge(df1, df2, on="Period")

# NOTE: Below works, but why relative to the one above?
df1 = pd.DataFrame({"Period": [2012]})
df2 = pd.DataFrame({"Period": [2012], "Stocks": [2227.093846]})
print(df1, type(df1))
print(df2, type(df1))
pd.merge(df1, df2, on="Period")


,Geolocation,Commodity,Period,Volume,Retail Price,Wholesale Price,Farmgate Price,Area Harvested,CPI All Income,CPI Bottom 30,Value of Production
0,cordillera administrative region (car),Rice,2012 Q1,56655.00,35.226667,32.166667,16.210000,14029.00,84.421871,83.394609,7573.0
1,region i (ilocos region),Rice,2012 Q1,313643.00,29.416667,31.533333,17.200000,67944.00,77.630850,78.960939,30010.0
2,region ii (cagayan valley),Rice,2012 Q1,463414.00,31.926667,30.670000,16.203333,118916.00,82.072230,84.694894,39075.0
3,region iii (central luzon),Rice,2012 Q1,628879.00,30.800000,32.796667,17.193333,131940.00,79.460275,79.446607,56876.0
4,region iv-a (calabarzon),Rice,2012 Q1,101543.00,30.676667,31.990000,14.473333,27805.00,80.329063,80.496204,5753.0
...,...,...,...,...,...,...,...,...,...,...,...
763,region x (northern mindanao),Rice,2023 Q4,229851.35,48.320000,46.096667,24.213333,46539.00,111.711589,109.703895,17614.0
764,region xi (davao region),Rice,2023 Q4,123140.78,49.313333,49.786667,21.996667,26758.00,110.723788,111.782338,9974.0
765,region xii (soccsksargen),Rice,2023 Q4,357203.95,46.030000,46.526667,21.650000,96022.00,103.242814,103.195269,25002.0
766,region xiii (caraga),Rice,2023 Q4,185738.35,52.763333,47.263333,20.846667,54426.00,104.740600,103.920879,10314.0


In [11]:
# RICE: Volume and Retail Price (Annual-Provincial)
# Volume of Rice
df1 = read_csv_to_df("datasets/agricultural-indicators/Volume_Rice and Corn_annual.csv")
df1 = df1[df1["Geolocation"].apply(is_province)]
df1 = df1[df1["Commodity"] == "Palay"]
df1["Commodity"] = df1["Commodity"].apply(lambda val: "Rice")
df1 = df1.melt(id_vars=["Commodity", "Geolocation"], value_vars=df1.columns[2:], var_name="Period", value_name="Volume")
df1 = swap_columns(df1, "Commodity", "Geolocation")

# Retail Price of Palay
df2 = read_csv_to_df("datasets/prices/prices_retail_2012-2023.csv")
df2 = df2[df2["Geolocation"].apply(is_province)]
df2.dropna(inplace=True)
df2 = get_annual_columns(df2, range(2012, 2024), 2)
df2.dropna(inplace=True)
df2 = df2[df2["Commodity"] == 'RICE, REGULAR-MILLED, 1 KG']
df2["Commodity"] = df2["Commodity"].apply(lambda val: "Rice")
df2 = df2.melt(["Geolocation", "Commodity"], df2.columns[2:], "Period", "Price")

# Volume and Price DataFrame
df = pd.merge(df1, df2)
df["Geolocation"] = df["Geolocation"].apply(lambda val: val.lstrip("."))
df = df[df["Volume"] != 0]
fig = px.scatter(df, x="Volume", y="Price", color="Geolocation")
fig.show()
df2

# # ------------------

# # RICE: Volume and Retail Price (Quarterly-Regional)
# df = read_csv_to_df("datasets/agricultural-indicators/Volume_Rice and Corn_quarterly.csv")
# df = df[df["Geolocation"].apply(is_region)]
# df = df[df["Commodity"] == "Palay"]
# df["Commodity"] = df["Commodity"].apply(lambda val: "Rice")
# df = df.melt(id_vars=["Commodity", "Geolocation"], value_vars=df.columns[2:], var_name="Period", value_name="Volume")
# df = swap_columns(df, "Commodity", "Geolocation")

# # Retail Price of Palay
# df2 = read_csv_to_df("datasets/prices/prices_retail_2012-2023.csv")
# df2 = df2[df2["Geolocation"].apply(is_region)]
# df2.dropna(inplace=True)
# df2 = get_quarter_columns(df2, range(2012, 2024), 2)
# df2.dropna(inplace=True)
# df2 = df2[df2["Commodity"] == 'RICE, REGULAR-MILLED, 1 KG']
# df2["Commodity"] = df2["Commodity"].apply(lambda val: "Rice")
# df2 = df2.melt(["Geolocation", "Commodity"], df2.columns[2:], "Period", "Price")

# # Volume and Price DataFrame
# df = pd.merge(df, df2)
# df["Geolocation"] = df["Geolocation"].apply(lambda val: val.lstrip("."))
# df = df[df["Volume"] != 0]
# fig = px.scatter(df, x="Volume", y="Price", color="Geolocation")
# # fig.show()

# # ------------------

# # RICE: Volume and Retail Price (Quarterly-Provincial)
# df = read_csv_to_df("datasets/agricultural-indicators/Volume_Rice and Corn_quarterly.csv")
# df = df[df["Geolocation"].apply(is_province)]
# df = df[df["Commodity"] == "Palay"]
# df["Commodity"] = df["Commodity"].apply(lambda val: "Rice")
# df = df.melt(id_vars=["Commodity", "Geolocation"], value_vars=df.columns[2:], var_name="Period", value_name="Volume")
# df = swap_columns(df, "Commodity", "Geolocation")

# # # Retail Price of Palay
# df2 = read_csv_to_df("datasets/prices/prices_retail_2012-2023.csv")
# df2 = df2[df2["Geolocation"].apply(is_province)]
# df2.dropna(inplace=True)
# df2 = get_quarter_columns(df2, range(2012, 2024), 2)
# df2.dropna(inplace=True)
# df2 = df2[df2["Commodity"] == 'RICE, REGULAR-MILLED, 1 KG']
# df2["Commodity"] = df2["Commodity"].apply(lambda val: "Rice")
# df2 = df2.melt(["Geolocation", "Commodity"], df2.columns[2:], "Period", "Price")

# # Volume and Price DataFrame
# df = pd.merge(df, df2)
# df["Geolocation"] = df["Geolocation"].apply(lambda val: val.lstrip("."))
# df = df[df["Volume"] != 0]
# fig = px.scatter(df, x="Volume", y="Price", color="Geolocation")
# fig.show()

# df.corr(numeric_only=True)

,Geolocation,Commodity,Period,Price
0,....Abra,Rice,2012,33.416667
1,....Apayao,Rice,2012,32.264167
2,....Benguet,Rice,2012,34.557500
3,....Ifugao,Rice,2012,43.230833
4,....Kalinga,Rice,2012,32.412500
...,...,...,...,...
871,....Surigao del Sur,Rice,2023,42.596667
872,....Basilan,Rice,2023,44.169167
873,....Lanao del Sur,Rice,2023,49.837500
874,....Maguindanao,Rice,2023,43.413333


In [ ]:
# Annual
df2 = read_csv_to_df("datasets/agricultural-indicators/Volume_Rice and Corn_annual.csv")
df2 = df2.melt(id_vars=["Geolocation", "Commodity"], value_vars=df2.columns[2:], var_name="Period", value_name="Volume")
df_volume = df2[df2["Geolocation"] == "PHILIPPINES"]

df2 = read_csv_to_df("datasets/prices/prices_retail_2012-2023.csv")
df2 = df2[df2["Geolocation"].apply(is_region)]
df2.dropna(inplace=True)
df2 = get_annual_columns(df2, range(2012, 2024), 2)
df2.melt(id_vars=["Geolocation", "Commodity"], value_vars=df2.columns[2:], var_name="Year", value_name="Volume")
